In [1]:
import re
from dateutil.parser import parse
from bs4 import BeautifulSoup
from urllib import request
import time

def altnames(mydict):
    aux_mydict = mydict.copy()
    for elem in mydict.keys():
        if "/" in elem:
            for alt in elem.split("/"):
                aux_mydict[alt] = mydict[elem]
            del aux_mydict[elem]
    return aux_mydict  


############# To be modified #############
##currentdate=time.strftime('%Y-%m-%d')
currentdate='2019-01-01'
TOPIC = "[Cc]onference|[Rr]oadshow|[Ii]nvestor days?"
pagename="Rational"
ID="2414"
address="https://www.rational-online.com/en_in/Company/Investor_Relations/Financial_and_roadshow_calendar_"
##########################################

In [2]:
cities=dict([line.strip().split(";") for line in open("city.csv", encoding='ISO-8859-1').readlines()[1:]])

In [3]:
cities["1"]

'Amsterdam'

In [4]:
cities["4"]

'Zurich/Zürich'

In [5]:
pcities = dict(zip([x for x in cities.values()],cities.keys()))
pcities=altnames(pcities)

In [6]:
pcities["Amsterdam"]

'1'

In [7]:
pcities["Zurich"],pcities["Zürich"]

('4', '4')

In [8]:
sponsors=dict([line.strip().split(";") for line in open("sponsor.csv").readlines()[1:]])
psponsors = dict(zip(sponsors.values(),sponsors.keys()))
psponsors = altnames(psponsors)

In [9]:
#!pip install lxml

In [19]:
url=request.urlopen(address,timeout=None).read()
soup = BeautifulSoup(url,"lxml")

In [11]:
len(soup.find_all("table"))

4

In [21]:
tab = soup.find_all("table")[1]

In [22]:
tab


<table class="table-standard table-mobile" width="100%">
<tbody>
<tr>
<td class="m-head">Event</td>
<td>Preliminary Figures Fiscal Year 2019</td>
</tr>
<tr>
<td class="m-head">Location</td>
<td>Landsberg am Lech</td>
</tr>
<tr>
<td class="m-head">Date</td>
<td>06 Feb 2020</td>
</tr>
<tr><td class="t-border"></td><td class="t-border"></td></tr>
<tr>
<td class="m-head">Event</td>
<td>Financial Figures Fiscal Year 2019</td>
</tr>
<tr>
<td class="m-head">Location</td>
<td>Hamburg</td>
</tr>
<tr>
<td class="m-head">Date</td>
<td>17 Mar 2020</td>
</tr>
<tr><td class="t-border"></td><td class="t-border"></td></tr>
<tr>
<td class="m-head">Event</td>
<td>Telephone Conference Fiscal Year 2019</td>
</tr>
<tr>
<td class="m-head">Location</td>
<td>Hamburg</td>
</tr>
<tr>
<td class="m-head">Date</td>
<td>17 Mar 2020</td>
</tr>
<tr><td class="t-border"></td><td class="t-border"></td></tr>
<tr>
<td class="m-head">Event</td>
<td>Balance Sheet Press Conference Fiscal Year 2019</td>
</tr>
<tr>
<td class=

In [14]:
tab.find_all("span",{"class":"givacolor"})

[]

In [15]:
tab.find_all("td")

[<td class="m-head">Event</td>,
 <td>Preliminary Figures Fiscal Year 2019</td>,
 <td class="m-head">Location</td>,
 <td>Landsberg am Lech</td>,
 <td class="m-head">Date</td>,
 <td>06 Feb 2020</td>,
 <td class="t-border"></td>,
 <td class="t-border"></td>,
 <td class="m-head">Event</td>,
 <td>Financial Figures Fiscal Year 2019</td>,
 <td class="m-head">Location</td>,
 <td>Hamburg</td>,
 <td class="m-head">Date</td>,
 <td>17 Mar 2020</td>,
 <td class="t-border"></td>,
 <td class="t-border"></td>,
 <td class="m-head">Event</td>,
 <td>Telephone Conference Fiscal Year 2019</td>,
 <td class="m-head">Location</td>,
 <td>Hamburg</td>,
 <td class="m-head">Date</td>,
 <td>17 Mar 2020</td>,
 <td class="t-border"></td>,
 <td class="t-border"></td>,
 <td class="m-head">Event</td>,
 <td>Balance Sheet Press Conference Fiscal Year 2019</td>,
 <td class="m-head">Location</td>,
 <td>Hamburg</td>,
 <td class="m-head">Date</td>,
 <td>17 Mar 2020</td>,
 <td class="t-border"></td>,
 <td class="t-border"></t

In [16]:
tab.find_all("td")[1]

<td>Preliminary Figures Fiscal Year 2019</td>

In [17]:
tab.find_all("td")[1].text

'Preliminary Figures Fiscal Year 2019'

In [18]:
for row in tab:
    for e in row.recursiveChildGenerator():
        if isinstance(e,str):
            print("++++++++++++++++++")
            print(e)

AttributeError: 'NavigableString' object has no attribute 'recursiveChildGenerator'

In [46]:
row  = soup.find_all("table")[0].find_all("tr")[2]
row

<tr>
<td colspan="1" rowspan="1">Financial Figures Fiscal Year 2019</td>
<td colspan="1" rowspan="1">Hamburg</td>
<td colspan="1" rowspan="1">17 Mar 2020</td>
</tr>

In [47]:
line=[elem.text.strip() for elem in row.find_all("td")]
line

['Financial Figures Fiscal Year 2019', 'Hamburg', '17 Mar 2020']

In [48]:
line = "$".join(line)
line

'Financial Figures Fiscal Year 2019$Hamburg$17 Mar 2020'

In [55]:
m1 = re.search(TOPIC,line)
print(m1 != None)

False


In [32]:
m1.group(0)

'Conference'

In [33]:
r="(.*)\$(.*)\$(.*)"
m2=re.search(r,line)
m2

<re.Match object; span=(0, 57), match='Telephone Conference Fiscal Year 2019$Hamburg$17 >

In [34]:
m2.group(0),m2.group(1),m2.group(2),m2.group(3)

('Telephone Conference Fiscal Year 2019$Hamburg$17 Mar 2020',
 'Telephone Conference Fiscal Year 2019',
 'Hamburg',
 '17 Mar 2020')

In [37]:
date=m2.group(3)
date

'17 Mar 2020'

In [38]:
parse(date,dayfirst=True).strftime('%Y-%m-%d')

'2020-03-17'

In [39]:
parse('10 January 2019',dayfirst=True).strftime('%Y-%m-%d')

'2019-01-10'

In [40]:
r="\d{1,2}(/\d{1,2})? [A-Z][a-z]+ \d{4}"

In [41]:
dm=re.search(r,date)
dm

<re.Match object; span=(0, 11), match='17 Mar 2020'>

In [42]:
firstPart=dm.group(0)
firstPart = re.sub("/\d{1,2}","",firstPart)
firstPart

'17 Mar 2020'

In [43]:
dm.group(1)

In [44]:
if dm.group(1):
    print(parse(firstPart).strftime('%Y-%m-%d')+dm.group(1)) 

In [45]:
x = 1.10 * 10**2
y = .085 * 10**2
x - y 

101.50000000000001